In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_excel('4aFeaturesAsap-2.xlsx')

In [4]:
df.columns

Index([               'Unnamed: 0.1',                  'Unnamed: 0',
                          'essay_id',                   'essay_set',
                             'essay',                      'ai_llm',
                      'ai_generated',                 'word_tokens',
                   'sentence_tokens',      'lemmatized_word_tokens',
                  'total_word_count',             'avg_word_length',
               'avg_sentence_length',                         'TTR',
                   'stop_word_count',           'unique_word_count',
                         'word_freq',                 'bigram_freq',
                      'trigram_freq',             'rare_word_count',
             'avg_sentence_length.1',        'avg_parse_tree_depth',
        'parse_tree_depth_variation', 'avg_adjectives_per_sentence',
          'avg_adverbs_per_sentence',      'avg_verbs_per_sentence',
            'avg_nouns_per_sentence',                             0,
               'flesch_reading_eas

In [5]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Initialize the model
rf = RandomForestClassifier()

features = df.drop(['ai_generated',
                    'Unnamed: 0.1',
                    'Unnamed: 0',
                    'essay',
                    'essay_id',
                    'essay_set',
                    'ai_llm',
                    'word_tokens',
                    'sentence_tokens',
                    'lemmatized_word_tokens',
                    'total_word_count',
                    'avg_word_length',
                    'avg_sentence_length',
                    "parse_tree_depth_variation",
                    0,
                    'rare_word_count',
                    'avg_sentence_length.1',
                    'word_freq',
                    'bigram_freq',
                    'trigram_freq'], axis=1)  # drop the target column
labels = df['ai_generated']

# Train the model
rf.fit(features, labels)

# Get feature importances
importances = rf.feature_importances_

# Create a DataFrame for visualization
feature_list = pd.DataFrame({'feature': features.columns, 'importance': importances})

# Sort DataFrame by importance
feature_list = feature_list.sort_values('importance', ascending=False)

# Print the top 10 features
print(feature_list.head(10))

                       feature  importance
1              stop_word_count    0.229650
2            unique_word_count    0.170188
0                          TTR    0.112531
10                  smog_index    0.085572
7       avg_nouns_per_sentence    0.070421
8          flesch_reading_ease    0.067156
3         avg_parse_tree_depth    0.051870
6       avg_verbs_per_sentence    0.046777
9   flesch_kincaid_grade_level    0.037155
11          sentiment_polarity    0.036790


In [6]:
# Select top 10 features
top_features = feature_list.head(10)

# Get the names of the top 10 features
top_feature_names = top_features['feature'].values

# Create a DataFrame containing only the top 10 features
selected_features_df = df[top_feature_names]


In [7]:
# Assuming `selected_features_df` is your DataFrame containing only the top 10 features
selected_features = selected_features_df

# Prepare input data for the model
X = selected_features.values.reshape(-1, 10, 1)  # reshape for CNN
Y = df['ai_generated'].values

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# Import necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense

# Define CNN model
model = Sequential()
model.add(Conv1D(32, 2, activation='relu', input_shape=(10, 1)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [8]:
# Train the model
model.fit(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), batch_size=32)

Epoch 1/50
904/904 [==============================] - 5s 3ms/step - loss: 0.4417 - accuracy: 0.8008 - val_loss: 0.3907 - val_accuracy: 0.8293
Epoch 2/50
904/904 [==============================] - 3s 3ms/step - loss: 0.3598 - accuracy: 0.8480 - val_loss: 0.3177 - val_accuracy: 0.8712
Epoch 3/50
904/904 [==============================] - 2s 3ms/step - loss: 0.3384 - accuracy: 0.8574 - val_loss: 0.3026 - val_accuracy: 0.8740
Epoch 4/50
904/904 [==============================] - 3s 4ms/step - loss: 0.3359 - accuracy: 0.8598 - val_loss: 0.3838 - val_accuracy: 0.8352
Epoch 5/50
904/904 [==============================] - 2s 3ms/step - loss: 0.3240 - accuracy: 0.8639 - val_loss: 0.2993 - val_accuracy: 0.8784
Epoch 6/50
904/904 [==============================] - 2s 3ms/step - loss: 0.3210 - accuracy: 0.8665 - val_loss: 0.4825 - val_accuracy: 0.7722
Epoch 7/50
904/904 [==============================] - 2s 3ms/step - loss: 0.3167 - accuracy: 0.8686 - val_loss: 0.3142 - val_accuracy: 0.8681
Epoch 

In [9]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict the values from the validation dataset
Y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Compute the confusion matrix
confusion_mtx = confusion_matrix(Y_test, Y_pred)

# Print the confusion matrix
print('Confusion Matrix')
print(confusion_mtx)

# Print the classification report
print('Classification Report')
print(classification_report(Y_test, Y_pred))


445/445 [==============================] - 2s 5ms/step
Confusion Matrix
[[6249  855]
 [ 618 6511]]
Classification Report
              precision    recall  f1-score   support

           0       0.91      0.88      0.89      7104
           1       0.88      0.91      0.90      7129

    accuracy                           0.90     14233
   macro avg       0.90      0.90      0.90     14233
weighted avg       0.90      0.90      0.90     14233



In [10]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define a function to create model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Conv1D(64, 2, activation='relu', input_shape=(10, 1)))  # Increased complexity
    model.add(Conv1D(64, 2, activation='relu'))  # Additional convolutional layer for increased complexity
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)




<ipython-input-10-84df76281e4a>:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [11]:
# Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)


In [12]:

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.903035 using {'batch_size': 40, 'epochs': 100}


In [15]:
# Generate predictions
Y_pred = (grid_result.predict(X_test) > 0.5).astype("int32")

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix")
print(conf_matrix)

# Calculate classification report
class_report = classification_report(Y_test, Y_pred)
print("Classification Report")
print(class_report)


445/445 [==============================] - 2s 3ms/step
Confusion Matrix
[[6633  471]
 [ 854 6275]]
Classification Report
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      7104
           1       0.93      0.88      0.90      7129

    accuracy                           0.91     14233
   macro avg       0.91      0.91      0.91     14233
weighted avg       0.91      0.91      0.91     14233



In [22]:
from tensorflow.keras.layers import Dropout

# Define CNN model
model = Sequential()
model.add(Conv1D(64, 2, activation='relu', input_shape=(7, 1)))  # Increased filters
model.add(Conv1D(32, 2, activation='relu'))  # Additional Convolutional layer
model.add(Flatten())
model.add(Dense(128, activation='relu'))  # Increased neurons
model.add(Dropout(0.5))  # Added dropout for regularization
model.add(Dense(64, activation='relu'))  # Additional Dense layer
model.add(Dropout(0.5))  # Added dropout for regularization
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), batch_size=40)


Epoch 1/100


ValueError: ignored

In [19]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict the values from the validation dataset
Y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Compute the confusion matrix
confusion_mtx = confusion_matrix(Y_test, Y_pred)

# Display the confusion matrix
print('Confusion Matrix')
print(confusion_mtx)

# Compute and display the classification report
print('Classification Report')
print(classification_report(Y_test, Y_pred))


445/445 [==============================] - 2s 4ms/step
Confusion Matrix
[[6400  704]
 [ 722 6407]]
Classification Report
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      7104
           1       0.90      0.90      0.90      7129

    accuracy                           0.90     14233
   macro avg       0.90      0.90      0.90     14233
weighted avg       0.90      0.90      0.90     14233



In [21]:
# Save the model
model.save('my_tuned_model.h5')


NameError: ignored